# Recursive Workshop Machine

This "machine" is made up of several functions that:

* **generates an image** from a prompt, and shows you the `revised_prompt` OpenAI *actually* uses under the hood
* **describes your image** using OpenAI's Vision API.
* **sends all this data to Airtable** (our databse) for analysis and printing

You can continue pushing this process towards a feedback loop or an [exquisite corpse](https://en.wikipedia.org/wiki/Exquisite_corpse)-like composition if you wish by duplicating and altering these cells however you'd like.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## set up packages and api variables

just installing all the packages and setting up api keys here.

### installing dependencies

In [ ]:
!pip install openai pyairtable



### defining keys and utils

In [ ]:
import textwrap
from google.colab import userdata

def print_wrapped(text, width=80):
    wrapped_text = textwrap.fill(text, width=width)
    print(wrapped_text)

OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
AIRTABLE_API_KEY=userdata.get('AIRTABLE_API_KEY')


## define image_from_prompt function

`prompt` is just a string that will serve as the image prompt

In [ ]:
from openai import OpenAI
import os
from IPython.display import Image, display, Markdown
from pyairtable import Api
import time

def image_from_prompt(prompt):

  ## create the image

  client = OpenAI(
      api_key = OPENAI_API_KEY
  )
  imageresponse = client.images.generate(
    model="dall-e-3",
    quality="hd",
    prompt=prompt,
    n=1,
    size="1024x1024"
  )

  image_url=imageresponse.data[0].url
  revised_prompt=imageresponse.data[0].revised_prompt

  ## display it in colab

  display(Image(url=image_url, width=500))
  display(Markdown("## prompt"))
  print_wrapped(prompt)
  display(Markdown("## revised_prompt"))
  print_wrapped(revised_prompt)

  ## send it to Airtable database

  base_id = userdata.get('airtable_base_id')
  table_name = "Images"

  api = Api(AIRTABLE_API_KEY)
  table = api.table(base_id, table_name)

  record_data = {
      "InitialURL": image_url,
      "ImageAttachment": [{"url": image_url}],
      "Name": prompt[:20].replace(" ", "-") + "-" + str(int(time.time())),
      "Prompt": prompt,
      "RevisedPrompt": revised_prompt
  }

  try:
      created_record=table.create(record_data)
      record_id = created_record['id']
      record_data['record_id'] = record_id
      print("Record created successfully in Airtable. It should be at this url: https://airtable.com/appxAzTlapU1rYEpi/tbl4TwRY585LzTnpK/viwkh8LS9ViJQzXtd?blocks=hide")
      return record_data
  except Exception as e:
      print(f"An error occurred: {e}")




## define description_from_image function

`image_data` is a dict that has this form (from previous function):
```
{
    "InitialURL": image_url,
    "ImageAttachment": [{"url": image_url}],
    "Name": prompt[:20].replace(" ", "-") + "-" + str(int(time.time())),
    "Prompt": prompt,
    "RevisedPrompt": revised_prompt,
    "record_id": [AIRTABLE RECORD ID]
}
```

In [ ]:

def description_from_image(image_data, vision_prompt):
  api = Api(AIRTABLE_API_KEY)
  client = OpenAI(api_key=OPENAI_API_KEY)

  base_id = userdata.get('airtable_base_id')
  table_name = "Images"

  api = Api(AIRTABLE_API_KEY)
  table = api.table(base_id, table_name)

  response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
      {
        "role": "user",
        "content": [
          {"type": "text", "text": vision_prompt},
          {
            "type": "image_url",
            "image_url": {
              "url": image_data["InitialURL"],
            },
          },
        ],
      }
    ],
    max_tokens=300,
  )
  vision_description = response.choices[0].message.content

  display(Markdown("## vision_description"))
  print_wrapped(vision_description)

  try:
      record_id = image_data['record_id']
      table.update(record_id, {'Description': vision_description})
      print("Record updated successfully. Should be visible in gallery here: https://airtable.com/appxAzTlapU1rYEpi/shrXDjwVGag2NEFHV")
      return vision_description
  except Exception as e:
      print(f"Failed to update record: {e}")



## run once

In [ ]:
image = image_from_prompt("a vision of an eclipse")
description = description_from_image(image, "what's in this image?")

## do recursive things



In [ ]:
image_1 = image_from_prompt("Create an image of a saloon girl from western american history")
description_1 = description_from_image(image_1, "describe this image as if you were a grizzled old miner")

image_2 = image_from_prompt("create a tapestry artwork based off of this image prompt: " + description_1)
description_2 = description_from_image(image_2, "describe this image as if you were telling an old german fairy tale")

image_3 = image_from_prompt("make me a vivid fairy tale story scene of this image prompt: " + description_2)
description_3 = description_from_image(image_3, "describe this image as if you were a standup comedian")

image_4 = image_from_prompt("make me an epic realistic reinactment of this image prompt: " + description_3)
description_4 = description_from_image(image_4, "describe this image with intense levels of detail and vigor")

image_5 = image_from_prompt("use the following as the prompt for an image, but intensify it in every respect as much as you can: " + description_4)
description_5 = description_from_image(image_5, "describe this image as if it were a religious text")
